**teneva** (examples)

This python package, named teneva (**ten**sor **eva**luation), provides very compact implementation for the multidimensional cross approximation algorithm in the tensor-train (TT) format. This package also contains a function for quickly calculating the values of the constructed low-rank tensor approximation, as well as a number of auxiliary useful utilities.

> See [github repo](https://github.com/AndreiChertkov/teneva) for more details and the License file.

---

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

# Example for tt-cross (TODO)

In [2]:
def f(x):
    y = np.sum([x[i]**(i+1) for i in range(d)])
    y = np.sin(y)**2 / (1 + y**2)
    return  y

def f_vect(X):
    """Naively vectorized model function."""
    return np.array([f(x) for x in X])

In [3]:
N = [9, 8, 8, 9, 8]  # Shape of the tensor
d = len(N)           # Dimension of the problem
M = 10000            # Number of test cases
nswp     = 10        # Sweep number
eps      = 1.E-6     # Desired accuracy
kickrank = 1         # Cross parameter
rf       = 1         # Cross parameter

In [4]:
X_tst = np.vstack([np.random.choice(N[i], M) for i in range(d)]).T
Y_tst = np.array([f(x) for x in X_tst])

In [5]:
Y0 = teneva.rand(N, 2)
teneva.show(Y0)

  9  8  8  9  8 
 / \/ \/ \/ \/ \
 1  2  2  2  2  1 



In [6]:
t = tpc()

Y = teneva.cross(f_vect, Y0, nswp, kickrank, rf)
teneva.show(Y)

Y = teneva.truncate(Y, eps)
teneva.show(Y)

t = tpc() - t
print(f'Build time (sec) : {t:-8.3f}')

  9  8  8  9  8 
 / \/ \/ \/ \/ \
 1 10 31 31  8  1 

  9  8  8  9  8 
 / \/ \/ \/ \/ \
 1  9 19 17  6  1 

Build time (sec) :    4.555


In [7]:
t = tpc()

get = teneva.getter(Y)

t = tpc() - t
print(f'Comp time  (sec) : {t:-8.3f}')

Comp time  (sec) :    0.622


In [8]:
t = tpc()

Z = np.array([get(x) for x in X_tst])

t = tpc() - t

e = np.linalg.norm(Z - Y_tst) / np.linalg.norm(Y_tst)

print(f'Check time (sec) : {t:-8.3f}')
print(f'Time 1-calc (ms) : {t/len(Z) * 1000:-8.3f}')
print(f'Relative error   : {e:-8.2e}')
print(f'Erank is         : {teneva.erank(Y):-8.2f}')

Check time (sec) :    0.436
Time 1-calc (ms) :    0.044
Relative error   : 4.54e-07
Erank is         :    13.80


## Compare with ttpy

In [9]:
import tt
from tt.cross.rectcross import cross as ttpy_cross

In [10]:
t = tpc()

x0 = tt.tensor.from_list(Y0)
Y = ttpy_cross(
    f_vect, x0=x0, nswp=nswp, eps=1.E-16, eps_abs=0., kickrank=kickrank, rf=rf, verbose=False)
Y = Y.round(eps)
Y = tt.tensor.to_list(Y)  

t = tpc() - t
print(f'Build time (sec) : {t:-8.3f}')

Build time (sec) :    3.109


In [11]:
get = teneva.getter(Y)
Z = np.array([get(x) for x in X_tst])
e = np.linalg.norm(Z - Y_tst) / np.linalg.norm(Y_tst)

print(f'Relative error   : {e:-8.2e}')
print(f'Erank is         : {teneva.erank(Y):-8.2f}')

Relative error   : 4.54e-07
Erank is         :    13.80


# Example for maxvol

In [12]:
from maxvolpy.maxvol import maxvol as base_maxvol

In [13]:
N = 5000
r = 50
A = 2. * np.random.rand(N, r) - 1.
e = 1.01
K = 500

In [14]:
t = tpc()
I, B = teneva.maxvol(A, e, K)
t = tpc() - t

In [15]:
t0 = tpc()
I0, B0 = base_maxvol(A, tol=e, max_iters=K)
t0 = tpc() - t0

In [16]:
ei = np.max(np.abs(I - I0))
eb = np.max(np.abs(B - B0))

In [17]:
print(f'Time own   : {t:-8.4f}')
print(f'Time base  : {t0:-8.4f}')
print(f'Error I    : {ei:-8.2e}')
print(f'Error B    : {eb:-8.2e}')
print(f'Det own    : {np.linalg.det(A[I, :]):-8.2e}')
print(f'Det base   : {np.linalg.det(A[I0, :]):-8.2e}')
print(f'Max B own  : {np.max(np.abs(B)):-8.2e}')
print(f'Max B base : {np.max(np.abs(B0)):-8.2e}')

Time own   :   0.2763
Time base  :   0.0220
Error I    : 0.00e+00
Error B    : 3.05e-15
Det own    : 1.54e+27
Det base   : 1.54e+27
Max B own  : 1.00e+00
Max B base : 1.00e+00


In [18]:
tr, ei, eb = [], [], []
for i in range(10):
    A = 2. * np.random.rand(N, r) - 1.
    t = tpc(); I, B = teneva.maxvol(A, e, K); t = tpc() - t
    t0 = tpc(); I0, B0 = base_maxvol(A, tol=e, max_iters=K); t0 = tpc() - t0
    tr.append(t / t0)
    ei.append(np.max(np.abs(I - I0)))
    eb.append(np.max(np.abs(B - B0)))

print(f'Mean (time own / time base) : {np.mean(tr):-8.3f}')
print(f'Max error for I             : {np.max(ei):-8.2e}')
print(f'Max error for B             : {np.max(eb):-8.2e}')

Mean (time own / time base) :   20.369
Max error for I             : 0.00e+00
Max error for B             : 4.66e-15


# Example for rect_maxvol

In [19]:
from maxvolpy.maxvol import rect_maxvol as base_rect_maxvol

In [20]:
N = 5000
r = 50
A = 2. * np.random.rand(N, r) - 1.
e = 1.01
N_min = r + 10
N_max = r + 30

# Special function syntax from maxvolpy package:
maxK = N_max           
min_add_K = N_min - r
minK = None

In [21]:
t = tpc()
I, B = teneva.rect_maxvol(A, e, N_min, N_max)
t = tpc() - t

In [22]:
t0 = tpc()
I0, B0 = base_rect_maxvol(A, e, maxK, min_add_K, minK)
t0 = tpc() - t0

In [23]:
ei = np.max(np.abs(I - I0))
eb = np.max(np.abs(B - B0))

In [24]:
print(f'Time own   : {t:-8.4f}')
print(f'Time base  : {t0:-8.4f}')
print(f'Error I    : {ei:-8.2e}')
print(f'Error B    : {eb:-8.2e}')
print(f'Max B elem : {np.max(np.abs(B)):-8.2e}')

Time own   :   0.4266
Time base  :   0.0420
Error I    : 0.00e+00
Error B    : 1.62e-15
Max B elem : 1.00e+00


In [25]:
tr, ei, eb = [], [], []
for i in range(10):
    A = 2. * np.random.rand(N, r) - 1.
    t = tpc()
    I, B = teneva.rect_maxvol(A, e, N_min, N_max)
    t = tpc() - t
    t0 = tpc()
    I0, B0 = base_rect_maxvol(A, e, maxK, min_add_K, minK)
    t0 = tpc() - t0
    tr.append(t / t0)
    ei.append(np.max(np.abs(I - I0)))
    eb.append(np.max(np.abs(B - B0)))
    
print(f'Mean (time own / time base) : {np.mean(tr):-8.3f}')
print(f'Max error for I             : {np.max(ei):-8.2e}')
print(f'Max error for B             : {np.max(eb):-8.2e}')

Mean (time own / time base) :   15.484
Max error for I             : 0.00e+00
Max error for B             : 2.56e-15


# Example for cross 2D (draft)

In [26]:
M = 1000
N = 500
r = 100
idx = np.random.choice(r, size=M)
A = np.random.randn(r, N)[idx]
# A = np.random.randn(M, r) @ np.random.randn(r, N)

In [27]:
t = tpc()
U1, S1, V1 = np.linalg.svd(A)
t = tpc() - t

A1 = U1[:, :r] @ np.diag(S1[:r]) @ V1[:r, :]
e = np.linalg.norm(A - A1) / np.linalg.norm(A)

print(f'Error                   : {e:-8.2e}')
print(f'Time (sec)              : {t:-8.4f}')

Error                   : 2.73e-15
Time (sec)              :   0.1237


In [28]:
t = tpc()
I, J = None, np.arange(r)
for q in range(5):
    I, B = teneva.maxvol(A[:, J], e=1.01, K=500)
    J, B = teneva.maxvol(A[I, :].T, e=1.01, K=500)

C, U, R = A[:, J], np.linalg.inv(A[np.ix_(I, J)]), A[I, :]
t = tpc() - t

A2 = C @ U @ R
# e = np.linalg.norm(A - A2) / np.linalg.norm(A)
# e = np.linalg.norm((A - A2).ravel()  , np.inf)
e = np.max(np.abs(A - A2))

print(f'Error                   : {e:-8.2e}')
print(f'Time (sec)              : {t:-8.4f}')

Error                   : 1.79e-14
Time (sec)              :   0.1299


In [29]:
M = 1000
N = 2000
r = 20
e = []
for i in range(100):
    A = np.random.randn(M, r) @ np.random.rand(r, N)
    I = np.random.choice(M, size=r, replace=False)
    J = np.random.choice(N, size=r, replace=False)
    C = A[:, J]
    U = np.linalg.inv(A[np.ix_(I, J)])
    R = A[I, :]
    B = C @ U @ R
    e.append(np.max(np.abs(A-B)))
print(f'Error max : {np.max(e):-8.2e}')

Error max : 4.63e-10


---